In [1]:
import orjson
import os
import pandas as pd
from tqdm import tqdm
import orjson

In [2]:
tier = 'diamond'
output_dir = "/home/piddle/hdd/matches"
target_dir = os.path.join(output_dir, tier)

In [ ]:
path = os.path.join(target_dir, os.listdir(target_dir)[6])
file = orjson.loads(open(path, 'rb').read())
file

In [4]:
all_column=['match_id', 'game_length_second', 'summoner_id', 'summoner_level', 'champion_id', 'team_key', 'position',
             'trinket_item', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'rune_0', 'rune_1', 'spell_0', 'spell_1',
             'champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
             'total_damage_taken', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'time_ccing_others',
             'vision_score', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place',
             'turret_kill', 'kill', 'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal', 'result']
    
cols1 = ['champion_id', 'team_key', 'position', 'trinket_item']

stat_cols = ['champion_level', 'damage_self_mitigated', 'damage_dealt_to_objectives', 'damage_dealt_to_turrets',
            'total_damage_taken', 'total_damage_dealt', 'total_damage_dealt_to_champions', 'time_ccing_others',
            'time_ccing_others', 'vision_wards_bought_in_game', 'sight_wards_bought_in_game', 'ward_kill', 'ward_place',
            'turret_kill', 'kill', 'death', 'assist', 'neutral_minion_kill', 'gold_earned', 'total_heal']

In [5]:
def parse_match(file_iter) -> pd.DataFrame:
    data_chunk = []
    for n, file in enumerate(tqdm(file_iter)):
        with open(file.path, 'rb') as f:
            try:
                json_data = orjson.loads(f.read())
            except Exception as e:
                print('Error reading {}: {}'.format(file.path, e))
                continue

        for match in json_data:
            for participant in match['participants']:
                data= {}
                data['match_id'] = match['id']
                data['game_length_second'] = match['game_length_second']

                data['summoner_id'] = participant['summoner']['summoner_id']
                data['summoner_level'] = participant['summoner']['level']

                for col in cols1:
                    data[col] = participant[col]

                for i, item in enumerate(participant['items']):
                    data[f'item_{i}'] = item

                data['rune_0'] = participant['rune']["primary_rune_id"]
                data['rune_1'] = participant['rune']["secondary_page_id"]
                data['spell_0'] = participant['spells'][0]
                data['spell_1'] = participant['spells'][1]

                stats = participant['stats']
                for col in stat_cols:
                    data[col] = stats[col] / match['game_length_second']

                data['vision_score'] = stats['vision_score']
                data['result'] = stats['result']

                data_chunk.append(data)

        if n % 20000 == 0:
            df = pd.DataFrame(data_chunk, columns=all_column)
            data_chunk = []
            yield df

    else:
        df = pd.DataFrame(data_chunk, columns=all_column)
        yield df

In [ ]:
file_iter = os.scandir(target_dir)
participants = pd.DataFrame(columns=all_column)
participants.to_csv(f'../data/{tier}_matches.csv', mode='w', index=False)

for df in parse_match(file_iter):
    pass
    df.to_csv(f'../data/{tier}_matches.csv', mode='a', index=False, header=False)

In [3]:
df = pd.read_csv(f'../data/{tier}_matches.csv')
df

,match_id,game_length_second,summoner_id,summoner_level,champion_id,team_key,position,trinket_item,item_0,item_1,...,ward_kill,ward_place,turret_kill,kill,death,assist,neutral_minion_kill,gold_earned,total_heal,result
0,CNrwP4AK2XcH9xkZsQERnIA14QTmSpQM8kL5ePISXRw=,1558,21lcFv4Ie7itx4Z_ZlRJldeznHupGH5FVvnP5skekdH4qA,405,43,BLUE,SUPPORT,3364,2055,4642,...,0.003851,0.015404,0.000000,0.000642,0.003851,0.006418,0.000000,4.329268,0.324134,LOSE
1,CNrwP4AK2XcH9xkZsQERnIA14QTmSpQM8kL5ePISXRw=,1558,uZArvleduZDIFa2HoELS9ry_8CEZihDnIeSWfWPXOJI_RA,508,78,RED,TOP,3364,3076,2033,...,0.000642,0.005135,0.001284,0.002567,0.002567,0.004493,0.000000,6.781772,5.948652,WIN
2,CNrwP4AK2XcH9xkZsQERnIA14QTmSpQM8kL5ePISXRw=,1558,k9YtdiLXh7-ksTN0-u2TEEdVeH5vcUXcUSbwDXmDKpHhC8k,544,64,RED,JUNGLE,3340,6610,3071,...,0.003209,0.005135,0.000000,0.001926,0.001284,0.010911,0.094994,6.657253,12.137997,WIN
3,CNrwP4AK2XcH9xkZsQERnIA14QTmSpQM8kL5ePISXRw=,1558,f4Eiet39ObQ6sAMxlW6Rd6TWhQQHnuMr9l4t0paSRxhct6Y,373,34,BLUE,MID,3364,1056,6657,...,0.003209,0.004493,0.000642,0.001284,0.003851,0.001284,0.000000,5.648267,1.522465,LOSE
4,CNrwP4AK2XcH9xkZsQERnIA14QTmSpQM8kL5ePISXRw=,1558,3P--2tW5s9t95ZqK0bCyeCruC7A5VUEPdOucmq914rrEOjA,208,234,BLUE,JUNGLE,3364,2055,6610,...,0.005777,0.003209,0.000000,0.004493,0.002567,0.001926,0.069961,7.041078,11.892811,LOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245986,CNrwP4AK2XfZva7fLpZRRbXi3axVPsqjqMfc7-GAIlg=,1873,RXZcz6YcEkqnXGooXr58DAdybStLunhBKtbmQI4cfKB_SZU,255,111,BLUE,SUPPORT,3364,3070,0,...,0.002670,0.019754,0.000000,0.000000,0.004271,0.007475,0.000000,4.251468,1.022958,LOSE
5245987,CNrwP4AK2XfZva7fLpZRRbXi3axVPsqjqMfc7-GAIlg=,1873,GBCd7g6u5KWlmMRGjYY_jsU7RzMSk1yDMp4iKFIv01LEKWU,621,143,RED,SUPPORT,3364,3871,3116,...,0.006407,0.016017,0.000534,0.000534,0.002670,0.012280,0.000000,5.271757,1.964762,WIN
5245988,CNrwP4AK2XfZva7fLpZRRbXi3axVPsqjqMfc7-GAIlg=,1873,DGXN1i6k8RV4TUlWWjhzhLUwSlFrmoddTP5Fx4vrpgmuUPo,92,203,RED,JUNGLE,3364,2055,3072,...,0.006941,0.005339,0.001068,0.006407,0.003737,0.003203,0.112653,8.369461,10.849439,WIN
5245989,CNrwP4AK2XfZva7fLpZRRbXi3axVPsqjqMfc7-GAIlg=,1873,SpE-OJYXk7VMKE77fvq_IeGUdSP4wYRFfO_Hc2zMK1IQUj4,116,74,BLUE,TOP,3363,3116,3157,...,0.000534,0.005339,0.000534,0.001068,0.001068,0.000534,0.000000,6.611319,2.053924,LOSE


In [6]:
sum_count = df['summoner_id'].value_counts()
sum_count = sum_count[sum_count >= 10]
sum_count

summoner_id
AkEZVjwHY2PfU67psVrKCQ55y6gmZMav8E9448cju0CAhQ                638
xXN0X7CzdAdciTBszU3d3xUYbIwWK3f_DeOllRDQI2V80iI               539
hQ-r4brv3eIKKGDpSzvdUgJ89pE2DfBSnmE_mrTdXv30VtIQyNQLFMs0Wg    537
V5RdGfh9kz5h1jMza4byREX2mmJhcAR0Oj7mZtZtvYPvKfhXm1ri91K3FQ    536
lI2BAkJpMdpIQXUWNkEMliZGIUPpYc9c1DCyuQGMngNrfU4               525
                                                             ... 
hR9T3NQzmPjJkRlU8qyXDU3QPDjgoN1ydXp5gt2_HNKgfi7dZIaGBbKbNw     10
efkrSQ2LCrR5e2I-nYuSPKfDEXzusjJQAvLMOoIf8jX4w6B_BZYkfCtrgg     10
PxTQuCB6XHRL7beD6cSLLpWsKyaXzUQHkk6kAClaEYhvrQ                 10
T-IQpw4Ob6TtBd7IK8_Xsty6lxYOJqmCGtoH-RGVQnh0Dw                 10
3QDJHQkHIIM9aJP5AvVEeyCGtJpUAYrzeklkgxdqsYjxF8HRAy6NK53plA     10
Name: count, Length: 100319, dtype: int64

In [23]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn

import itertools

In [24]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.combinations = [list(element) for element in itertools.combinations(range(5), 2)]
        self.combinations_len = len(self.combinations)

    def __getitem__(self, idx):
        df_idx = idx // self.combinations_len
        combination_idx = idx % self.combinations_len

        x = self.df.iloc[df_idx, self.combinations[combination_idx]]
        x = torch.tensor(x, dtype=torch.int32, device='cuda')

        y = self.df.iloc[df_idx, -1]
        y = torch.tensor(y, dtype=torch.float64, device='cuda')

        return x, y
        
    def __len__(self):
        return len(self.combinations) * len(self.df)

In [26]:
class SimilarityModel(nn.Module):
    def __init__(self, config):
        super(SimilarityModel, self).__init__()
        self.embedding = nn.Embedding(config['n_layers'], config['emb_size'])

        # categorical
        self.cate_emb = nn.Embedding(cfg.total_cate_size, cfg.emb_size, padding_idx=0)
        self.cate_proj = nn.Sequential(
            nn.Linear(cfg.emb_size * cfg.cate_col_size, cate_col_hidden_size),
            nn.LayerNorm(cate_col_hidden_size),
        )

        # continuous
        self.cont_bn = nn.BatchNorm1d(cfg.cont_col_size)
        self.cont_emb = nn.Sequential(
            nn.Linear(cfg.cont_col_size, cont_col_hidden_size),
            nn.LayerNorm(cont_col_hidden_size),
        )

        # combination
        self.comb_proj = nn.Sequential(
            nn.ReLU(),
            nn.Linear(cfg.hidden_size*4, cfg.hidden_size),
            nn.LayerNorm(cfg.hidden_size)
        )


    def forward(self, input):
        input = input.transpose_(0, 1)
        embedded = self.embedding(input)
        A = embedded[0]
        B = embedded[1]
        output = F.cosine_similarity(A, B, dim=1)     

        return output

In [31]:
df = pd.read_csv(f'../data/{tier}_matches.csv')

config = {}
config['n_layers'] = int(df.drop(columns='result').max().max() + 1)
config['emb_size'] = 3

dataset = MyDataset(df)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = SimilarityModel(config).to('cuda')

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

model.train()
for x, y in tqdm(loader):
    optimizer.zero_grad()
    output = model(x)

    # print(output, y)
    loss = loss_fun(output, y)
    loss.backward()
    optimizer.step()

  0%|          | 0/2521284 [00:00<?, ?it/s]/tmp/ipykernel_34020/378348694.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = torch.tensor(x, dtype=torch.int32, device='cuda')
100%|██████████| 2521284/2521284 [2:06:43<00:00, 331.59it/s]  


In [33]:
import joblib

joblib.dump(model, 'cossim.joblib')

['cossim.joblib']